In [170]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon
import branca.colormap as cm
from folium.plugins import TimestampedGeoJson

In [2]:
bikedata=pd.read_json('/Users/alexanderlindell/Documents/Programmering /Python/Sthlm-EbikeVis-/stations2-copy.json')

In [161]:
df={'date':[],'station':[],'occupancy':[],'capacity':[],'long':[],'lat':[]}
df

{'date': [],
 'station': [],
 'occupancy': [],
 'capacity': [],
 'long': [],
 'lat': []}

In [162]:
test=bikedata.iloc[0:10]
test

,stations
003b4566-1a1d-4aa5-9b2c-35972a0ac219,"{'id': '003b4566-1a1d-4aa5-9b2c-35972a0ac219',..."
004f0690-df63-4b3c-9949-11261b199720,"{'id': '004f0690-df63-4b3c-9949-11261b199720',..."
014cf32d-c432-48e8-a32f-e8c85c143f5a,"{'id': '014cf32d-c432-48e8-a32f-e8c85c143f5a',..."
10b15a0e-4e7d-436b-bd49-b8e793a99a72,"{'id': '10b15a0e-4e7d-436b-bd49-b8e793a99a72',..."
159e8622-7572-43fb-a47e-44c9b4d272f8,"{'id': '159e8622-7572-43fb-a47e-44c9b4d272f8',..."
1850edf4-c993-4882-a88f-35da22a4f0c2,"{'id': '1850edf4-c993-4882-a88f-35da22a4f0c2',..."
1bbd622b-3658-4b38-910e-d8dff887bcdd,"{'id': '1bbd622b-3658-4b38-910e-d8dff887bcdd',..."
1c299c9c-aa80-43a1-ac97-3d8effeb10cf,"{'id': '1c299c9c-aa80-43a1-ac97-3d8effeb10cf',..."
1c887652-e924-4f34-b568-e4567ed10926,"{'id': '1c887652-e924-4f34-b568-e4567ed10926',..."
1dd1330c-a172-4abf-83ba-37362e42e2d4,"{'id': '1dd1330c-a172-4abf-83ba-37362e42e2d4',..."


In [37]:
for index,row in test.iterrows():
    for item in row:
        for cell in item['data']: 
            print(cell)

{'occupancy': 4, 'capacity': 12, 'date': 1652995800.689033}
{'occupancy': 4, 'capacity': 12, 'date': 1652996701.325262}
{'occupancy': 4, 'capacity': 12, 'date': 1652997602.2256892}
{'occupancy': 4, 'capacity': 12, 'date': 1652998502.9561138}
{'occupancy': 6, 'capacity': 12, 'date': 1652999403.751864}
{'occupancy': 6, 'capacity': 12, 'date': 1653000304.692656}
{'occupancy': 4, 'capacity': 12, 'date': 1653001205.721483}
{'occupancy': 4, 'capacity': 12, 'date': 1653002106.724565}
{'occupancy': 5, 'capacity': 12, 'date': 1653003007.931624}
{'occupancy': 5, 'capacity': 12, 'date': 1653003909.17551}
{'occupancy': 5, 'capacity': 12, 'date': 1653004810.546846}
{'occupancy': 5, 'capacity': 12, 'date': 1653005711.897152}
{'occupancy': 5, 'capacity': 12, 'date': 1653006614.082196}
{'occupancy': 5, 'capacity': 12, 'date': 1653007515.717197}
{'occupancy': 5, 'capacity': 12, 'date': 1653008417.214858}
{'occupancy': 5, 'capacity': 12, 'date': 1653009318.9012449}
{'occupancy': 5, 'capacity': 12, 'date

In [163]:
for index,row in test.iterrows():
    for item in row:
        for cell in item['data']: 
            #print(cell['date'])
            df['occupancy'].append(cell['occupancy'])
            df['capacity'].append(cell['capacity'])
            df['date'].append(cell['date'])
            df['station'].append(item['id'])
            #print(cell['date'],item['coord']['lon'],item['coord']['lat'])
            #df['geometry'].append(Point(item['coord']['lon'],item['coord']['lat']) )
            df['long'].append(item['coord']['lon'])
            df['lat'].append(item['coord']['lat'])


In [164]:
print(len(df['station']))
print(len(df['date']))
print(len(df['occupancy']))
print(len(df['capacity']))
print(len(df['long']))
print(len(df['lat']))

400
400
400
400
400
400


In [165]:
data=pd.DataFrame.from_dict(df)
data['date']= pd.to_datetime(df['date'],unit='s')
data=data.set_index('date')

In [166]:
data=data.sort_values(by='date')
data.head()


,station,occupancy,capacity,long,lat
date,,,,,
2022-05-19 21:30:00.689033031,003b4566-1a1d-4aa5-9b2c-35972a0ac219,4,12,17.993884,59.324599
2022-05-19 21:30:00.689033031,10b15a0e-4e7d-436b-bd49-b8e793a99a72,0,10,18.070001,59.330138
2022-05-19 21:30:00.689033031,159e8622-7572-43fb-a47e-44c9b4d272f8,1,12,18.066735,59.330670
2022-05-19 21:30:00.689033031,014cf32d-c432-48e8-a32f-e8c85c143f5a,0,16,18.068239,59.343017
2022-05-19 21:30:00.689033031,1850edf4-c993-4882-a88f-35da22a4f0c2,0,10,18.074349,59.329609


In [97]:
gdf=gpd.GeoDataFrame(data,geometry=data['geometry'])

In [98]:
gdf

,station,occupancy,capacity,geometry
date,,,,
2022-05-19 21:30:00.689033031,003b4566-1a1d-4aa5-9b2c-35972a0ac219,4,12,POINT (17.99388 59.32460)
2022-05-19 21:30:00.689033031,004f0690-df63-4b3c-9949-11261b199720,3,16,POINT (18.07380 59.33074)
2022-05-19 21:45:01.325262070,003b4566-1a1d-4aa5-9b2c-35972a0ac219,4,12,POINT (17.99388 59.32460)
2022-05-19 21:45:01.325262070,004f0690-df63-4b3c-9949-11261b199720,3,16,POINT (18.07380 59.33074)
2022-05-19 22:00:02.225689173,003b4566-1a1d-4aa5-9b2c-35972a0ac219,4,12,POINT (17.99388 59.32460)
...,...,...,...,...
2022-05-20 06:46:15.477914810,003b4566-1a1d-4aa5-9b2c-35972a0ac219,5,12,POINT (17.99388 59.32460)
2022-05-20 07:01:18.965840816,004f0690-df63-4b3c-9949-11261b199720,4,16,POINT (18.07380 59.33074)
2022-05-20 07:01:18.965840816,003b4566-1a1d-4aa5-9b2c-35972a0ac219,5,12,POINT (17.99388 59.32460)


In [167]:
colormap = cm.StepColormap(colors=['#7FFF00','#FFA500','#FF7F00','#FF4500','#8B0000'], vmin=0,vmax=1)
colormap

In [169]:
map = folium.Map(location = [59.32760990395156, 18.06760960579676], tiles='Stamen Toner' , zoom_start = 13)
map
colormap = cm.StepColormap(colors=['#7FFF00','#FFA500','#FF7F00','#FF4500','#8B0000'], vmin=0,vmax=1)
for index,row in data.iterrows():

    folium.CircleMarker(location=[row['lat'],row['long']],radius=2,color=colormap(row['occupancy']//row['capacity'])).add_to(map)
map.add_child(colormap)
display(map)

In [ ]:
def create_geojson_features(df):
    features = []
    
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['long'],row['lat']]
            },
            'properties': {
                'time': pd.to_datetime(row['hour'], unit='h').__str__(),
                'style': {'color' : ''},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': row['fillColor'],
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': row['count'] + 5
                }
            }
        }
        features.append(feature)
    return features
start_geojson[0]

In [ ]:
start_geojson = create_geojson_features(start)
start_geojson[0]

In [ ]:
TimestampedGeoJson(start_geojson,
                  period = 'PT1H',
                  duration = 'PT1M',
                  transition_time = 1000,
                  auto_play = True).add_to(map)

map
